# Test Conversation

Simple notebook to test conversations:
1. Paste exported conversation
2. Set model
3. Run cell

In [ ]:
# One-time setup
!pip install requests python-dotenv -q

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

# ============================================================
# PASTE EXPORTED CONVERSATION HERE
# ============================================================
conversation = {
  "messages": [
    {"role": "system", "content": "System prompt here"},
    {"role": "user", "content": "User message here"}
  ]
}

# ============================================================
# SET MODEL HERE
# ============================================================
model = "anthropic/claude-sonnet-4.5"
# Options:
#   "anthropic/claude-sonnet-4.5" (pro tier storytelling, best reasoning)
#   "anthropic/claude-haiku-4.5" (free tier storytelling, fast)
#   "google/gemini-2.5-pro" (pro tier management)
#   "google/gemini-2.5-flash" (free tier management, fast)

# ============================================================
# RUN (automatic with high reasoning effort)
# ============================================================
print(f"🚀 Testing {model} with {len(conversation['messages'])} messages...\n")

payload = {
    "model": model,
    "input": [
        {"type": "message", "role": msg["role"], "content": [{"type": "input_text", "text": msg["content"]}]}
        for msg in conversation["messages"]
    ],
    "reasoning": {"effort": "high"},
    "max_output_tokens": 8000,
    "stream": False,
}

response = requests.post(
    "https://openrouter.ai/api/v1/responses",
    headers={"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"},
    json=payload,
)

result = response.json()

# Show reasoning
for item in result.get('output', []):
    if item.get('type') == 'reasoning':
        print("🧠 REASONING:")
        for i, step in enumerate(item.get('summary', []), 1):
            print(f"  {i}. {step}")
        print()

# Show response
for item in result.get('output', []):
    if item.get('type') == 'message':
        for content in item.get('content', []):
            if content.get('type') == 'output_text':
                print("📝 RESPONSE:\n")
                print(content.get('text', ''))
                print("\n" + "="*80)

# Show usage
if 'usage' in result:
    usage = result['usage']
    total = usage.get('total_tokens', 0)
    reasoning = usage.get('output_tokens_details', {}).get('reasoning_tokens', 0)
    print(f"\n📊 Tokens: {total} total ({reasoning} reasoning)")